# Ã–n EÄŸitim - EÅŸli SÄ±nÄ±flandÄ±rma

Bu Ã§alÄ±ÅŸmada ilk Ã¶n eÄŸitim notebook'unda olduÄŸu gibi maskeli dil modeli eÄŸitilmekte ve ek olarak emoji sÄ±nÄ±flandÄ±rmayÄ± ise eÅŸli bir ÅŸekilde yapÄ±yoruz.

Bu yaklaÅŸÄ±m yine BERT'te olan bir sonraki cÃ¼mleyi sÄ±nÄ±flandÄ±rmaya (next sentence prediction) benziyor. Burada ikili bir ÅŸekilde eÅŸleÅŸtirdiÄŸimiz emojileri siliniÅŸ twitlerden, bu iki twitin aynÄ± emojiyi iÃ§erip iÃ§ermediÄŸini tahmin eden bir eÅŸli sÄ±nÄ±flandÄ±rma gÃ¶revi oluÅŸturmaktayÄ±z.

Ã–rneÄŸin,
Twit 1: Bu tam bir hayal kÄ±rÄ±klÄ±ÄŸÄ± ðŸ˜¦
Twit 2: Ã‡ok mutluyum ðŸ™ƒ
Twit 3: SÄ±nav iÃ§in Ã§ok stresliyim ðŸ˜¦

iÃ§in

*("Bu tam bir hayal kÄ±rÄ±klÄ±ÄŸÄ±", "Ã‡ok mutluyum")* ikilisi farklÄ± kategorilerden emojiler iÃ§erdiÄŸi iÃ§in bunlarÄ±n farklÄ± olduÄŸunu modelimizin bu ÅŸekilde (0) ayÄ±rt etmesini istiyoruz.

*("Bu tam bir hayal kÄ±rÄ±klÄ±ÄŸÄ±", "SÄ±nav iÃ§in Ã§ok stresliyim")* ikilisi aynÄ± kategoriden emojiler iÃ§erdiÄŸi iÃ§in bunlarÄ±n aynÄ± olduÄŸunu modelimizin bu ÅŸekilde (1) ayÄ±rt etmesini istiyoruz.


Gerekli kÃ¼tÃ¼phaneler

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from transformers import AdamW, AutoConfig, AutoModelForPreTraining, AutoTokenizer
import torch
import numpy as np
import random
import math
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
from random import shuffle
from random import random as rand
from torch.nn import CrossEntropyLoss
import sys
import os
from random import randrange
from datetime import datetime
from utils import get_emoji_pairs 

In [2]:
learning_rate = 0.00003
wd = 0.1
loss_multiplier = 5
transformer_name = "mbert"

# Transformerdaki CLS ve SEP taglerinin tanÄ±mlanmasÄ±
cls_tag = 101
sep_tag = 102
if transformer_name=="distilberturk":
    cls_tag = 2
    sep_tag = 3

embedding_layer = True
last_free_layer = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())
print(torch.cuda.is_available())

cuda:0
1
True


In [3]:
if transformer_name=="mbert":
    config = AutoConfig.from_pretrained('bert-base-multilingual-cased')
    config.output_hidden_states = True
elif transformer_name=="distilberturk":
    config = AutoConfig.from_pretrained('dbmdz/distilbert-base-turkish-cased')
    config.output_hidden_states = True

MaskelenmiÅŸ Dil Modeli'nin eklendiÄŸi modelin tanÄ±mlanmasÄ± 

In [4]:
class Combined_Net(nn.Module):
    def __init__(self):
        super(Combined_Net, self).__init__()
        if transformer_name=="distilberturk":
            self.net_bert = AutoModelForPreTraining.from_config(config).from_pretrained('dbmdz/distilbert-base-turkish-cased', output_hidden_states=True)
        elif transformer_name=="mbert":
            self.net_bert = AutoModelForPreTraining.from_config(config).from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
        unfrozen_layers = ["cls", "pooler"]
        if embedding_layer:
            unfrozen_layers.append('embedding')
        
        for idx in range(last_free_layer, 12):
            if transformer_name=="distilberturk":
                unfrozen_layers.append('transformer.layer.'+str(idx))
            elif transformer_name=="mbert":
                unfrozen_layers.append('encoder.layer.'+str(idx))
            
            
        print(unfrozen_layers)
        for name, param in self.net_bert.named_parameters():
            if not any([layer in name for layer in unfrozen_layers]):
                print("[FROZE]: %s" % name)
                param.requires_grad = False
            else:
                print("[FREE]: %s" % name)
                param.requires_grad = True

        self.fc1 = nn.Linear(768, 1)

    def forward(self, input_ids, input_attention, input_types, input_ids_masked, input_attention_masked, input_types_masked):
        if transformer_name=="mbert":
            _, _, x  = self.net_bert(input_ids, attention_mask=input_attention, token_type_ids=input_types)
            probs, _, _  = self.net_bert(input_ids_masked, attention_mask=input_attention_masked, token_type_ids=input_types_masked)
        elif transformer_name=="distilberturk":
            _, x  = self.net_bert(input_ids, attention_mask=input_attention)
            probs, _  = self.net_bert(input_ids_masked, attention_mask=input_attention_masked)
    

        #Getting head
        x = x[-1][:,0,:]

        x = self.fc1(x)
        return x, probs

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

try:
    combined_net.apply(weight_reset)
    print('Combined Net resetlendi.')
except: 
    pass

combined_net = Combined_Net().to(device)
print('Combined Net tanÄ±mlandÄ±.')

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['cls', 'pooler', 'embedding', 'encoder.layer.0', 'encoder.layer.1', 'encoder.layer.2', 'encoder.layer.3', 'encoder.layer.4', 'encoder.layer.5', 'encoder.layer.6', 'encoder.layer.7', 'encoder.layer.8', 'encoder.layer.9', 'encoder.layer.10', 'encoder.layer.11']
[FREE]: bert.embeddings.word_embeddings.weight
[FREE]: bert.embeddings.position_embeddings.weight
[FREE]: bert.embeddings.token_type_embeddings.weight
[FREE]: bert.embeddings.LayerNorm.weight
[FREE]: bert.embeddings.LayerNorm.bias
[FREE]: bert.encoder.layer.0.attention.self.query.weight
[FREE]: bert.encoder.layer.0.attention.self.query.bias
[FREE]: bert.encoder.layer.0.attention.self.key.weight
[FREE]: bert.encoder.layer.0.attention.self.key.bias
[FREE]: bert.encoder.layer.0.attention.self.value.weight
[FREE]: bert.encoder.layer.0.attention.self.value.bias
[FREE]: bert.encoder.layer.0.attention.output.dense.weight
[FREE]: bert.encoder.layer.0.attention.output.dense.bias
[FREE]: bert.encoder.layer.0.attention.output.LayerNorm.weig

In [5]:
if transformer_name=="distilberturk":
    tokenizer = AutoTokenizer.from_pretrained('dbmdz/distilbert-base-turkish-cased')
elif transformer_name=="mbert":
    tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Mandalina Emoji Verileri Ã¼zerinden otomatik olarak ikililerin oluÅŸturulmasÄ±

In [6]:
df = get_emoji_pairs("../Veriler/emoji.csv")
tokenizer_vocabs = list(tokenizer.vocab.keys())

MaskelenmiÅŸ Dil Modeli iÃ§in otomatik bir ÅŸekilde eÄŸitim verisinin oluÅŸturulmasÄ± ve Ã¶znitelik Ã§Ä±karma kodlarÄ±

In [7]:
def to_masked(original_sent):

    max_pred = 20
    mask_prob = 0.15
    max_len = 256
    first_part = list(tokenizer.encode(original_sent[0], add_special_tokens=False))
    second_part = list(tokenizer.encode(original_sent[1], add_special_tokens=False))
    feature = [cls_tag]+first_part+[sep_tag]+second_part+[sep_tag]
    if len(feature)>256:
        print('Error in ',original_sent)
        feature = feature[:255]+[sep_tag]
    mid_index = len(first_part)+2
    input_type = [0]*mid_index+(len(feature)-mid_index)*[1]
    
    masked_tokens = []
    masked_pos = []
    tokens = tokenizer.convert_ids_to_tokens(feature)
    input_mask = [1]*len(tokens)
    n_pred = min(max_pred, max(1, int(round(len(tokens)*mask_prob))))
    cand_pos = [i for i, token in enumerate(tokens)
                if token != '[CLS]' and token != '[SEP]' and token != '[unused5]']
    shuffle(cand_pos)

    for pos in cand_pos[:n_pred]:
        masked_tokens.append(tokens[pos])
        masked_pos.append(pos)
        if rand() < 0.8: # 80%
            tokens[pos] = '[MASK]'
        elif rand() < 0.5: # 10%
            tokens[pos] = tokenizer_vocabs[random.randrange(len(tokenizer_vocabs))]

            
#     masked_weights = [1]*len(masked_tokens)
    
    masked_lm_labels = [-100]*max_len
    for pos in masked_pos:
        masked_lm_labels[pos] = feature[pos]
    # Token Indexing
    input_ids = tokenizer.encode(tokens, add_special_tokens=False)
#     masked_ids = tokenizer.encode(masked_tokens, add_special_tokens=False)
    n_pad = max_len - len(input_ids)
    input_ids.extend([0]*n_pad)
    input_mask.extend([0]*n_pad)
    input_type.extend([0]*n_pad)
    
        # Zero Padding for masked target
#     if max_pred > n_pred:
#         n_pad = max_pred - n_pred
#         masked_ids.extend([0]*n_pad)
#         masked_pos.extend([0]*n_pad)
#         masked_weights.extend([0]*n_pad)

    return torch.tensor(input_ids), torch.tensor(input_mask), torch.tensor(input_type), torch.tensor(masked_lm_labels)

def to_masked_all(X):
    ids = []
    mask = []
    types = []
    labels = []
    for _, original_sent in X:
        try:
            input_ids, input_mask, input_type, masked_lm_labels = to_masked(original_sent)
        except:
            print(original_sent)
            raise Exception
        ids.append(input_ids)
        mask.append(input_mask)
        types.append(input_type)
        labels.append(masked_lm_labels)
    return torch.stack(ids),torch.stack(mask),torch.stack(types),torch.stack(labels)

def to_id(first_text, second_text):
    ids_1 = tokenizer.encode(first_text, add_special_tokens=False)
    ids_2 = tokenizer.encode(second_text, add_special_tokens=False)
    return torch.tensor([cls_tag]+ids_1+[sep_tag]+ids_2+[sep_tag])

def feat_ext(data):
    features = []
    attention_masks = []
    type_ids = []
    max_len = 256
    for input_ids, _ in data:
        first_ind = list(input_ids).index(sep_tag)+1
        if transformer_name=="distilberturk":
            type_id = []
        elif transformer_name=="mbert":
            type_id = torch.cat((torch.tensor([0]*(first_ind)), torch.tensor([1]*(len(input_ids)-first_ind)),torch.tensor([0]*(max_len-len(input_ids)))), 0)
        attention_mask = torch.cat((torch.tensor([1.0]*(len(input_ids))), torch.tensor([0.0]*(max_len-len(input_ids)))), 0)
        input_ids = torch.cat((input_ids, torch.tensor([0]*(max_len-len(input_ids)))), 0)        
        attention_masks.append(attention_mask)
        features.append(input_ids)
        type_ids.append(torch.tensor(type_id))
    return torch.stack(features),torch.stack(attention_masks),torch.stack(type_ids)

def feat_ext_batch(subset_df):
    X = []
    y = []
    ct = 0
    for idx, row in subset_df.iterrows():
        id_seq = to_id(row['text1'], row['text2'])
        if len(id_seq)>=256:
            ct+=1
            continue
        X.append((id_seq, (row['text1'], row['text2'])))
        y.append([row['class']*1.0])
    X_feat, X_attention, X_types = feat_ext(X)
    X_feat_masked, X_attention_masked, X_types_masked, X_masked_lm_labels = to_masked_all(X)
    y = torch.tensor(y)
    return X_feat, X_attention, X_types, X_feat_masked, X_attention_masked, X_types_masked, X_masked_lm_labels, y



Emoji veri setinin eÄŸitim ve test kÄ±sÄ±mlarÄ±nÄ±n ayrÄ±lmasÄ±

In [8]:
msk = np.random.rand(len(df)) < 0.95
train_df = df[msk]
test_df = df[~msk].reset_index(drop=True)
msk = np.random.rand(len(train_df)) < 0.94
val_df = train_df[~msk].reset_index(drop=True)
train_df = train_df[msk].reset_index(drop=True)

Optimizer ve loss fonksiyonun belirlenmesi

In [9]:
criterion = nn.BCEWithLogitsLoss()
lm_criterion = CrossEntropyLoss()

optimizer = AdamW(combined_net.parameters(), lr=learning_rate,  correct_bias=False, weight_decay=wd)

Dil modeli ve emoji ikili sÄ±nÄ±flandÄ±rma iÃ§in tahminleri dÃ¶nen kod

In [10]:
def get_predictions(df):
    batch_size = 16
    lm_loss = 0
    cl_loss = 0
    with torch.no_grad():
        outputs = torch.tensor([], device='cpu')
        y_test = torch.tensor([], device='cpu')
        for idx in tqdm(range(math.ceil(len(df)/batch_size)), total=math.ceil(len(df)/batch_size)):
            inputs_0, input_attention, input_type, inputs_0_m, input_attention_m, input_type_m, input_masked_labels, y_test_sub = feat_ext_batch(df[idx*batch_size: min(len(df), (idx+1)*batch_size)])

            inputs_0 = inputs_0.to(device)
            input_attention = input_attention.to(device)
            input_type = input_type.to(device)
            inputs_0_m = inputs_0_m.to(device)
            input_attention_m = input_attention_m.to(device)
            input_type_m = input_type_m.to(device)

            o, probs = combined_net(inputs_0, input_attention, input_type, inputs_0_m, input_attention_m, input_type_m) 
            outputs = torch.cat((outputs, o.to('cpu')), 0)
            y_test = torch.cat((y_test, y_test_sub), 0)
            
            lm_loss += lm_criterion(probs.to('cpu').view(-1, config.vocab_size), input_masked_labels.view(-1))
            cl_loss += criterion(o.to('cpu'), y_test_sub)
        predicted_test = torch.sigmoid(outputs)>0.5
        total = y_test.size(0)
        correct = (predicted_test == y_test).sum().item()
        test_acc = correct/total

    test_f1 = f1_score(predicted_test.cpu(), y_test.cpu())
    return test_f1, test_acc, lm_loss, cl_loss

In [11]:
best_val_acc = 0
batch_size = 4
best_val_loss = np.inf
accumulation_steps = 64

for epoch in range(10):
    
    running_loss = 0.0
    running_cl_loss = 0.0
    running_lm_loss = 0.0
    total_loss = 0
    total_cl_loss = 0.0
    total_lm_loss = 0.0
    total = 0
    correct = 0
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    test_df = test_df.sample(frac=1).reset_index(drop=True)
    val_df = val_df.sample(frac=1).reset_index(drop=True)
    train_outputs = torch.BoolTensor([]).to(device)
    for idx in range(math.ceil(len(train_df)/batch_size)):
        inputs_0, input_attention, input_type, inputs_0_m, input_attention_m, input_type_m, input_masked_labels, labels = feat_ext_batch(train_df[idx*batch_size: min(len(train_df), (idx+1)*batch_size)])
        inputs_0 = inputs_0.to(device)
        input_attention = input_attention.to(device)
        input_type = input_type.to(device)
        inputs_0_m = inputs_0_m.to(device)
        input_attention_m = input_attention_m.to(device)
        input_type_m = input_type_m.to(device)
        input_masked_labels = input_masked_labels.to(device)
        labels = labels.to(device)
        
        outputs, probs = combined_net(inputs_0, input_attention, input_type, inputs_0_m, input_attention_m, input_type_m)
        
        lm_loss = lm_criterion(probs.view(-1, config.vocab_size), input_masked_labels.view(-1)) / accumulation_steps 

        predicted = torch.sigmoid(outputs)>0.5
        correct += (predicted == labels).sum().item()
        total+= len(labels)
        train_outputs = torch.cat((train_outputs, predicted), 0)
        # forward + backward + optimize
        cl_loss = criterion(outputs, labels) / accumulation_steps 
        
        loss = loss_multiplier*cl_loss + lm_loss
        loss.backward()
        
        if (idx+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()                           # Reset gradients tensors
#             scheduler.step()
        
        

        # print statistics
        running_loss += loss.item()
        running_cl_loss += cl_loss.item()
        running_lm_loss += lm_loss.item()
        if (idx+1) % accumulation_steps == 0:    # print every 2000 mini-batches
            print('[%d_%d-%d, %5d/%d] loss: %.3f lm_loss: %.3f cl_loss: %.3f  |  accuracy: %.3f' %
                  (epoch + 1, (idx + 1)//accumulation_steps,(idx + 1)%accumulation_steps, idx + 1, len(train_df)//batch_size, running_loss, running_lm_loss, running_cl_loss, correct/total))
            total_loss += running_loss
            total_cl_loss += running_cl_loss
            total_lm_loss += running_lm_loss
            running_loss = 0.0
            running_cl_loss = 0 
            running_lm_loss = 0
        

            
    train_acc = correct/total
    print(train_acc)
    
    test_f1, test_acc, test_lm_loss, test_cl_loss = get_predictions(test_df)
    test_loss = (loss_multiplier*test_cl_loss) + test_lm_loss
    val_f1, val_acc, val_lm_loss, val_cl_loss = get_predictions(val_df)
    val_loss = (loss_multiplier*val_cl_loss) + val_lm_loss
   
    if val_loss<best_val_loss:
        now = datetime.now()
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': combined_net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': total_loss
                }, f'../Models/{transformer_name}_pair_f1_{test_f1}_acc_{test_acc}_{epoch+1}.pt')
    print('Epoch: ',epoch+1)
    print(f'Loss: {total_loss}, LM Loss: {total_lm_loss}, CL Loss: {total_cl_loss}, Training accuracy:{train_acc}, Validation accuracy:{val_acc}, Test accuracy:{test_acc}')
    print(f'Val F1:{val_f1} \t Val Loss: {val_loss} Val CL Loss: {val_cl_loss} Val LM Loss: {val_lm_loss}')
    print(f'Test F1:{test_f1} \t Test Loss: {test_loss} Test CL Loss: {test_cl_loss}  LM Loss: {val_lm_loss}')

/home/akoksal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[1_1-0,    64/223284] loss: 7.220 lm_loss: 3.756 cl_loss: 0.693  |  accuracy: 0.512
[1_2-0,   128/223284] loss: 8.132 lm_loss: 4.644 cl_loss: 0.698  |  accuracy: 0.502
[1_3-0,   192/223284] loss: 7.536 lm_loss: 4.071 cl_loss: 0.693  |  accuracy: 0.488
[1_4-0,   256/223284] loss: 7.388 lm_loss: 3.897 cl_loss: 0.698  |  accuracy: 0.484
[1_5-0,   320/223284] loss: 7.289 lm_loss: 3.856 cl_loss: 0.686  |  accuracy: 0.502
[1_6-0,   384/223284] loss: 7.525 lm_loss: 3.842 cl_loss: 0.737  |  accuracy: 0.494
[1_7-0,   448/223284] loss: 7.173 lm_loss: 3.702 cl_loss: 0.694  |  accuracy: 0.495
[1_8-0,   512/223284] loss: 7.328 lm_loss: 3.843 cl_loss: 0.697  |  accuracy: 0.495
[1_9-0,   576/223284] loss: 7.149 lm_loss: 3.690 cl_loss: 0.692  |  accuracy: 0.498
[1_10-0,   640/223284] loss: 7.112 lm_loss: 3.650 cl_loss: 0.692  |  accuracy: 0.501
[1_11-0,   704/223284] loss: 7.153 lm_loss: 3.676 cl_loss: 0.695  |  accuracy: 0.500
[1_12-0,   768/223284] loss: 7.054 lm_loss: 3.589 cl_loss: 0.693  |  accur

KeyboardInterrupt: 